In [4]:
!pip install "snowflake-connector-python[pandas]"
!pip install "snowflake-connector-python[secure-local-storage,pandas]"
!pip install --upgrade snowflake-sqlalchemy
!pip install mysql-connector-python

  Obtaining dependency information for mysql-connector-python from https://files.pythonhosted.org/packages/d3/b8/6798a0f91e595c0704a94c1e32a00ca930f77cb8f0f96c7e4dad4f9db1fe/mysql_connector_python-8.1.0-cp311-cp311-win_amd64.whl.metadata
  Using cached mysql_connector_python-8.1.0-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
Using cached mysql_connector_python-8.1.0-cp311-cp311-win_amd64.whl (10.8 MB)


In [23]:
# MySQL
import mysql
import pandas as pd
import mysql.connector as connection
# Snowflake
from snowflake.sqlalchemy import URL
#import snowflake.connector
from sqlalchemy import create_engine
from snowflake.connector.pandas_tools import pd_writer


def fix_date_cols(df, tz='UTC'):
    cols = df.select_dtypes(include=['datetime64[ns]']).columns
    for col in cols:
        df[col] = df[col].dt.tz_localize(tz)
    return df

        
# MySQL
db_mysql = mysql.connector.connect(
    host="72.167.124.128",
    user="ecommerce_prd_admin",
    password="L0GM4CAdcMVNrMBFZKzYVGEZlbxQ",
    database="ecommerce_prd"
)


# Snowflake
engine = create_engine(URL(
    account = 'pypoxiq-om82528',
    user = 'BOOTCAMPDA910',
    password = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx',
    database = 'ecommerce',
    schema = 'RAW_MYSQL',
    warehouse = 'COMPUTE_WH',
    role='ACCOUNTADMIN',
    #session_parameters={}
))

connection = engine.connect()
results = connection.execute('select current_version()').fetchone()
print('Conectado a Snowflake v{}!'.format(results[0]) )

Conectado a Snowflake v7.34.0!


In [24]:
import warnings
warnings.filterwarnings("ignore")


# MySQL
query = '''SELECT
            table_schema
              , table_name
              , table_rows
            FROM information_schema.tables
            WHERE TABLE_TYPE = "BASE TABLE"
        '''
df = pd.read_sql(query, db_mysql)

In [25]:
with engine.connect() as con:
    for index, row in df.T.items():
        #read from mysql
        schema = row['table_schema']
        table = row['table_name']
        query = "SELECT * FROM {0}".format(table)
        print("loading...{0}".format(table))
        df_mysql = pd.read_sql(query, db_mysql)
        df_mysql = fix_date_cols(df_mysql)
        #write to snowflake
        df_mysql = df_mysql.reset_index(drop=True)
        df_mysql.columns = map(lambda x: str(x).upper(), df_mysql.columns)
        print("writing on Snowflake..".format(table))
        df_mysql.to_sql(name=table.lower(), con=con, index=False, if_exists='replace', method=pd_writer)
        print("-----------------------------")
db_mysql.close()

loading...customers
writing on Snowflake..
-----------------------------
loading...geolocation
writing on Snowflake..
-----------------------------
loading...order_items
writing on Snowflake..
-----------------------------
loading...order_payments
writing on Snowflake..
-----------------------------
loading...order_reviews
writing on Snowflake..
-----------------------------
loading...orders
writing on Snowflake..
-----------------------------
loading...product_category_name_translation
writing on Snowflake..
-----------------------------
loading...products
writing on Snowflake..
-----------------------------
loading...sellers
writing on Snowflake..
-----------------------------
